In [2]:
#!pip install annoy

# Load model
from model import Siamese
from process import load_data, load_json_config

import os

run_name = "edit_emb20_lay8"
k = 0
save_file = os.path.join("saves", f"{run_name}_k{k}_BEST")

json_file = os.path.join("configs", str(run_name) + ".json")
DATASET_CONFIG, TRAIN_CONFIG, MODEL_KWARGS = load_json_config(json_file)

_, model, _, _, _, _, _ = load_data(save_file, TRAIN_CONFIG, MODEL_KWARGS)

RuntimeError: Error(s) in loading state_dict for Siamese:
	Unexpected key(s) in state_dict: "preddef_linear.0.weight", "preddef_linear.0.bias", "preddef_linear.2.weight", "preddef_linear.2.bias", "preddef_linear.4.weight", "preddef_linear.4.bias", "preddef_linear.6.weight", "preddef_linear.6.bias". 

In [ ]:
import pandas as pd

dataset_name = "1800s_first_name_pairs.tsv"

dataset_path = os.path.join("data", dataset_name)

dataset = pd.read_csv(dataset_path, sep = "\t", names = ["name1", "name2"])
dataset = dataset.to_dict("records")